# PEtab v2 import

This notebook demonstrates how to import a [PEtab v2 problem](https://petab.readthedocs.io/en/latest/v2/documentation_data_format.html) for computing the objective function and gradient, or for simulating individual conditions.

In [ ]:
import amici
from amici import run_simulation
from amici.petab.petab_importer import *
from amici.petab.simulations import EDATAS, RDATAS
from amici.plotting import (
    plot_observable_trajectories,
    plot_state_trajectories,
)
from benchmark_models_petab import get_problem_yaml_path
from petab.v2 import Problem

In [ ]:
# Load a PEtab v2 problem
problem_id = "Brannmark_JBC2010"
get_problem_yaml_path(problem_id)
problem = Problem.from_yaml(
    get_problem_yaml_path(problem_id.replace("Benchmark-Models", "v2"))
)
problem

First, we create an AMICI model from the PEtab v2 problem. This will account for the observation model encoded in the PEtab problem, as well as for the different experimental conditions. This is important to keep in mind when using the model anything else than the PEtab-encoded experiments.

In [ ]:
importer = PetabImporter(problem)

In [ ]:
simulator = importer.create_simulator()

In [ ]:
# simulate all conditions encoded in the PEtab problem for which there are measurements
result = simulator.simulate()
assert all(r.status == amici.AMICI_SUCCESS for r in result[RDATAS]), (
    "Simulation failed."
)

In [ ]:
result

## Simulating individual conditions

It's also possible to simulate only specific experimental conditions encoded in the PEtab problem. The `ExperimentManager` takes care of creating `ExpData` the respective instances, and setting the condition-specific parameters, measurements, and initial states:

In [ ]:
model = importer.import_module().get_model()
# It's important to use the petab Problem from the importer, as it was modified to encode the experimental conditions.
em = ExperimentManager(model=model, petab_problem=importer.petab_problem)

In [ ]:
importer.petab_problem.experiments

In [ ]:
# TODO: this should (at least optionally) set the nominal parameter values

edata = em.create_edata(importer.petab_problem.experiments[-1].id)
em.apply_parameters(
    edata,
    dict(
        zip(
            importer.petab_problem.x_ids,
            importer.petab_problem.x_nominal,
            strict=True,
        )
    ),
)

In [ ]:
solver = model.create_solver()
rdata = run_simulation(model, solver, edata)
assert rdata.status == amici.AMICI_SUCCESS, "Simulation failed."

In [ ]:
plot_observable_trajectories(rdata, model=model, edata=edata)

In [ ]:
plot_state_trajectories(rdata, model=model)

In [ ]:
rdata.x

In [ ]:
rdata.status

In [ ]:
rdata = run_simulation(model, solver, result[EDATAS][-1])

In [ ]:
plot_observable_trajectories(rdata, model=model, edata=edata)

In [ ]:
plot_state_trajectories(rdata, model=model)